In [6]:
using SparseArrays

In [7]:
include("../../src/ocean_model.jl")
include("../../src/schemes/helmholtz_sparse.jl")

sp_solve_modified_helmholtz

In [8]:
domain = RectangularDomain(0, 1, 0, 1)
dx = 0.25
M = 4
P = 4
lambda = 0.0

f(x, y) = x*y

# Range includes extra index at each end for ghost cell.
xs = range(domain.x1 - dx, domain.x2, length=M+2)
ys = range(domain.y1 - dx, domain.y2, length=P+2)

-0.25:0.25:1.0

In [9]:
# Use - here to ensure the matrix is semi pos def.
A = -construct_spA(M+2, P+2, lambda, dx)
display(A)
b = inflate(f, xs, ys)
display(b)

b[:,1] = b[:,end-1] # LHS ghost column.
b[:,end] = b[:,2] # RHS ghost column.
b[1,:] = b[end-1,:] # Top ghost row.
b[end,:] = b[2,:] # Bottom ghost row.

display(b)

A \ vec(b)

36×36 SparseMatrixCSC{Float64, Int64} with 156 stored entries:
⎡⠻⣦⡀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⢄⠈⠛⣤⡀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠑⢄⠈⠻⣦⠀⠑⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠑⢄⠀⠻⣦⡀⠑⢄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠑⢄⠈⠛⣤⡀⠑⢄⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠑⢄⠈⠻⣦⠀⠑⢄⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠻⣦⡀⠑⢄⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠈⠛⣤⡀⠑⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠈⠻⣦⎦

6×6 Matrix{Float64}:
  0.0625  -0.0  -0.0625  -0.125  -0.1875  -0.25
 -0.0      0.0   0.0      0.0     0.0      0.0
 -0.0625   0.0   0.0625   0.125   0.1875   0.25
 -0.125    0.0   0.125    0.25    0.375    0.5
 -0.1875   0.0   0.1875   0.375   0.5625   0.75
 -0.25     0.0   0.25     0.5     0.75     1.0

6×6 Matrix{Float64}:
 0.5625  0.0  0.1875  0.375  0.5625  0.0
 0.0     0.0  0.0     0.0    0.0     0.0
 0.1875  0.0  0.0625  0.125  0.1875  0.0
 0.375   0.0  0.125   0.25   0.375   0.0
 0.5625  0.0  0.1875  0.375  0.5625  0.0
 0.0     0.0  0.0     0.0    0.0     0.0

36-element Vector{Float64}:
 -0.014613734535687846
 -0.011649344071375693
 -0.016083311486027995
 -0.020813835416538147
 -0.021009505918600242
 -0.007821751255210306
 -0.011649344071375696
 -0.01590033026378693
 -0.02015131645619816
 -0.02272502426152435
  ⋮
 -0.033195121588515716
 -0.03210406297228732
 -0.01343487935605893
 -0.007821751255210306
 -0.01027749910224098
 -0.013042058151101103
 -0.01491801477391894
 -0.013434879356058933
 -0.0067174396780294654

In [11]:
function main()
    H_1 = 1000.0
    H_2 = 2000.0
    beta = 2*10^-11
    Lx = 4000.0 # 4000 km
    Ly = 2000.0 # 2000 km
    dt = 1 # 30 minutes
    T = 2000
    U = 1.0 # Forcing term of top level.
    M = 100
    P = 100
    dx = 1.0
    beta = 1.0
    visc = 1.0
    
    simulation_name = "test2"
    file_name = "../../data/$simulation_name.jld"

    # M+2 and P+2 for ghost cells on each side.
    zeta = zeros(M+2, P+2, 2, 3)
    psi = zeros(M+2, P+2, 2, 3)

     # M x P x 2 x 3, 2 for each layer and 3 time points are kept for AB3.
    #  zeta::Array{Real, 4}    
    #  psi::Array{Real, 4}

    # Initialise the model.
    for z in 1:2
        a = rand(Float64, (M+2, P+2))
        zeta[:,:,z,1] = update_ghost_cells(a)
        b = rand(Float64, (M+2, P+2))
        psi[:,:,z,1] = update_ghost_cells(b)
    end    

    # Create the file and write the initial conditions.
    f = jldopen(file_name, "w") do file
        write(file, "zeta_0", zeta[:,:,:,1])
        write(file, "psi_0", psi[:,:,:,1])
    end 

    # Create the model.
    model = BaroclinicModel(H_1, H_2, beta, Lx, Ly, dt, T, U, M, P, dx, visc)

    for timestep in 1:T
        zeta = evolve_zeta(model, zeta, psi, timestep)
        psi = evolve_psi(model, zeta, psi)

        if timestep % 25 == 0
            f = jldopen(file_name, "r+") do file
                write(file, "zeta_$timestep", zeta[:,:,:,1])
                write(file, "psi_$timestep", psi[:,:,:,1])
            end
        end
    end
end

main()